#1) Load Data

In [ ]:
!pip install psycopg2-binary
# psycopg2 import
import psycopg2
import pandas as pd
# 연결정보 입력
connection_info = "host=147.47.200.145 dbname=teamdb12 user=team12 password=494196 port=34543" #34543
conn = psycopg2.connect(connection_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.3 MB/s eta 0:00:00


In [ ]:
df_jockey = pd.read_sql('SELECT * FROM public."l0_api_jockeyinfo"',conn)
df_horse = pd.read_sql('SELECT * FROM public."l0_api_racehorseinfo"',conn)
trainData = pd.read_sql('SELECT * FROM public."l1_crawl_race_result_detail"',conn)
testData = pd.read_sql('SELECT * FROM public."l1_crawl_race_plan_detail"',conn)

<ipython-input-2-bcf95320f685>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_jockey = pd.read_sql('SELECT * FROM public."l0_api_jockeyinfo"',conn)
<ipython-input-2-bcf95320f685>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_horse = pd.read_sql('SELECT * FROM public."l0_api_racehorseinfo"',conn)
<ipython-input-2-bcf95320f685>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trainData = pd.read_sql('SELECT * FROM public."l1_crawl_race_result_detail"',conn)
<ipython-input-2-bcf95320f685>:4: UserWarning: pandas only

# 2) Data Preprocess Methods

In [ ]:
def merge_all(data, jockey, horse, forTrain):
  # Merge1 (+ 기수정보)
  data.rename(columns = {'jkno' : 'jkNo'}, inplace = True) # match name
  data_jockey = jockey[['jkNo','ord1CntT','ord1CntY', 'ord2CntT', 'ord2CntY', 'ord3CntT', 'ord3CntY', 'rcCntT', 'rcCntY']]
  df_merge = pd.merge(data, data_jockey, left_on = 'jkNo', right_on = 'jkNo', how = 'left')
  # 경주마정보의 Column이름과 구분을 위해 변수명끝에 j삽입.
  name_dict = {'ord1CntT':'ord1cntt_j',
              'ord1CntY': 'ord1cnty_j',
              'ord2CntT': 'ord2cntt_j',
              'ord2CntY':'ord2cnty_j',
              'ord3CntT':'ord3cntt_j',
              'ord3CntY':'ord3cnty_j',
              'rcCntT' : 'rccntt_j',
              'rcCntY':'rccnty_j'}
  df_merge.rename(columns = name_dict, inplace = True)

  # Merge2 (+경주마정보)
  df_merge.rename(columns = {'hrno' : 'hrNo'}, inplace = True) # match name
  data_horse = horse[['hrNo', 'ord1CntT','ord1CntY', 'ord2CntT', 'ord2CntY', 'ord3CntT', 'ord3CntY', 'rcCntT', 'rcCntY']]
  df_merge = pd.merge(df_merge, data_horse, left_on = 'hrNo', right_on = 'hrNo', how = 'left')

  # Slice features that model uses.
  if forTrain:
    df_final = df_merge[['rcdate', 'rcno', 'ord','hrNo','jkNo', 'rctime', 'age',
                        'rating','wghr','sex','ord1cntt_j',
                        'ord1cnty_j', 'ord2cntt_j', 'ord2cnty_j', 'ord3cntt_j','ord3cnty_j',
                        'rccntt_j', 'rccnty_j', 'ord1CntT', 'ord1CntY','ord2CntT', 'ord2CntY',
                        'ord3CntT', 'ord3CntY', 'rcCntT', 'rcCntY']]
  else:
     df_final = df_merge[['rcdate', 'rcno','hrNo','jkNo', 'age',
                        'rating','wghr','sex','ord1cntt_j',
                        'ord1cnty_j', 'ord2cntt_j', 'ord2cnty_j', 'ord3cntt_j','ord3cnty_j',
                        'rccntt_j', 'rccnty_j', 'ord1CntT', 'ord1CntY','ord2CntT', 'ord2CntY',
                        'ord3CntT', 'ord3CntY', 'rcCntT', 'rcCntY']]

  return df_final

In [ ]:
from sklearn import preprocessing
def data_preProcess(data, forTrain):
  # 1)sex into one-hot-encoding
  data = pd.get_dummies(data, columns=['sex'])
  names_dummies = {'sex_거': 'sex_n', 'sex_수': 'sex_male', 'sex_암': 'sex_female'}
  data = data.rename(columns=names_dummies)

  if forTrain:
    data[['rcno', 'ord', 'rating']] = data[['rcno', 'ord', 'rating']].apply(pd.to_numeric, errors=  'coerce')
    # 2)convert rctime (target 분-> 초)
    minuite, sec = data['rctime'].str.split(':').str[0], data['rctime'].str.split(':').str[1].astype(float)
    minuite = minuite.replace('', 0).astype(int)
    data['rctime'] = minuite*60+sec
  else:
    data[['rcno', 'rating']] = data[['rcno', 'rating']].apply(pd.to_numeric, errors=  'coerce')

  # 3)거리 (distance 정보 필요.)
  # 4)NA Control
  data['rating'].fillna(data['rating'].mean(), inplace = True)
  data['wghr'].fillna(data['wghr'].mean(), inplace = True)
  idx1 = data.columns.get_loc('ord1cntt_j')
  idx2 = data.columns.get_loc('rcCntY')
  data.iloc[:, idx1:(idx2+1)] = data.iloc[:, idx1:(idx2+1)].fillna(0)

  # 7) Normalize
  lst_column_norm = ['age','rating','wghr']
  data[lst_column_norm] = preprocessing.scale(data[lst_column_norm])

  return data

# 3) Model Development

In [ ]:
df_train = merge_all(trainData, df_jockey, df_horse, forTrain = True)
df_train = data_preProcess(df_train, forTrain = True)

In [ ]:
df_train.shape

(120441, 28)

In [ ]:
df_train.describe()

,rcno,ord,rctime,age,rating,wghr,ord1cntt_j,ord1cnty_j,ord2cntt_j,ord2cnty_j,...,ord1CntY,ord2CntT,ord2CntY,ord3CntT,ord3CntY,rcCntT,rcCntY,sex_n,sex_male,sex_female
count,120441.000000,117927.000000,118112.000000,1.204410e+05,1.204410e+05,1.204410e+05,120441.000000,120441.000000,120441.000000,120441.000000,...,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000,120441.000000
mean,6.274400,6.184970,87.073013,-1.661302e-16,8.082327e-17,1.463078e-16,414.458781,23.636619,392.332752,22.652627,...,0.119527,2.087371,0.131384,2.101801,0.137138,22.180852,1.470745,0.229772,0.362974,0.407253
std,3.407542,3.412261,18.376825,1.000004e+00,1.000004e+00,1.000004e+00,485.830440,22.282124,414.449031,19.223156,...,0.452669,1.986143,0.477151,1.946919,0.505039,12.694217,3.223117,0.420688,0.480860,0.491325
min,1.000000,1.000000,58.300000,-1.502530e+00,-2.077268e+00,-9.093982e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,75.800000,-6.196947e-01,-5.262873e-01,-2.867076e-01,120.000000,2.000000,135.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,6.000000,83.200000,2.631404e-01,0.000000e+00,8.106864e-02,285.000000,22.000000,283.000000,20.000000,...,0.000000,2.000000,0.000000,2.000000,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,9.000000,90.700000,2.631404e-01,0.000000e+00,4.101316e-01,464.000000,38.000000,416.000000,41.000000,...,0.000000,3.000000,0.000000,3.000000,0.000000,29.000000,0.000000,0.000000,1.000000,1.000000
max,15.000000,16.000000,159.900000,5.560151e+00,5.936132e+00,2.849069e+00,2207.000000,92.000000,2046.000000,62.000000,...,5.000000,13.000000,5.000000,15.000000,7.000000,101.000000,19.000000,1.000000,1.000000,1.000000


In [ ]:
# erase horses that haven't reached the finish line.
df_train = df_train.dropna(axis = 0)
# Data Split
X_train = df_train.iloc[:,6:]
y_train = df_train.rctime

In [ ]:
# Model fit
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(max_features = 7, random_state = 494196)
model_rf.fit(X_train, y_train)

RandomForestRegressor(max_features=7, random_state=494196)

In [ ]:
#model save
import joblib
joblib.dump(model_rf, 'model_rf.pkl')

['model_rf.pkl']

# 4) Prediction

In [ ]:
#import rf model
import joblib
model_rf = joblib.load("model_rf.pkl")

In [ ]:
# load data
df_test = merge_all(testData, df_jockey, df_horse, forTrain = False)
df_test = data_preProcess(df_test, forTrain = False)
# list_date = to control date change in loop.
list_date = df_test.rcdate.unique().tolist()

In [ ]:
# Result Prediction
result_pred = pd.DataFrame({'Date':[], 'Round':[], 'FirstRank': [], 'SecondRank': [], 'ThirdRank':[]})
for i in range(len(list_date)):
  games = df_test[df_test['rcdate'] == list_date[i]] # games = 해당 날짜의 시행되는 모든 경기
  list_raceNum = games.rcno.unique().tolist()
  for j in range(len(list_raceNum)):
    data = games[games['rcno'] == int(list_raceNum[j])] # data = 해당 날짜의 개별 경기
    X_test = data.iloc[:,4:]
    data['record_pred'] = model_rf.predict(X_test)
    data['rank_pred'] = data['record_pred'].rank().astype(int)
    # 각 개별 경기의 1stRank, 2ndRank, 3rdRank prediction 값(hrNo) 저장.
    result_pred.loc[len(result_pred)] = [int(list_date[i]) ,int(list_raceNum[j]), data[data['rank_pred'] == 1]['hrNo'].values[0],
                        data[data['rank_pred'] == 2]['hrNo'].values[0],
                        data[data['rank_pred'] == 3]['hrNo'].values[0]]

In [ ]:
result_pred

,Date,Round,FirstRank,SecondRank,ThirdRank
0,20230527,1,0046360,0045313,0045621
1,20230527,2,0045976,0046290,0046012
2,20230527,3,0045821,0045339,0045232
3,20230527,4,0046351,0046536,0045225
4,20230527,5,0045802,0046025,0045755
5,20230527,6,0046043,0045881,0045957
6,20230527,7,0042055,0045130,0042533
7,20230527,8,0045856,0044049,0046511
8,20230527,9,0044869,0045826,0042577
9,20230527,10,0046513,0042159,0043513
